In [10]:
PROJECT_DIR=`git rev-parse --show-toplevel`
cd $PROJECT_DIR

threads=16

# Pull out AMR gene

In [14]:
conda run --name staramr staramr search -o staramr-out input/SRR1952908.fasta

2020-01-15 22:35:36 INFO: No --pointfinder-organism specified. Will not search the PointFinder databases
2020-01-15 22:35:36 INFO: No --plasmidfinder-database-type specified. Will search the entire PlasmidFinder database
2020-01-15 22:35:36 INFO: --output-dir set. All files will be output to [staramr-out]
2020-01-15 22:35:36 INFO: Will exclude ResFinder/PointFinder genes listed in [/home/CSCScience.ca/apetkau/miniconda3/envs/staramr/lib/python3.7/site-packages/staramr/databases/exclude/data/genes_to_exclude.tsv]. Use --no-exclude-genes to disable
2020-01-15 22:35:36 INFO: Making BLAST databases for input files
2020-01-15 22:35:36 INFO: Scheduling blasts and MLST for SRR1952908.fasta
2020-01-15 22:35:38 INFO: Finished. Took 0.04 minutes.
2020-01-15 22:35:38 INFO: Predicting AMR resistance phenotypes is enabled. The predictions are for microbiological resistance and *not* clinical resistance. These results are continually being improved and we welcome any feedback.
2020-01-15 22:35:38 IN

In [22]:
head staramr-out/hits/resfinder_SRR1952908.fasta -n 15 > staramr-out/aadA1.fasta
head staramr-out/aadA1.fasta

>aadA1_3_JQ414041 isolate: SRR1952908, contig: contig00030, contig_start: 5355, contig_end: 4564, database_gene_start: 1, database_gene_end: 792, hsp/length: 792/792, pid: 100.00%, plength: 100.00%
ATGAGGGAAGCGGTGATCGCCGAAGTATCGACTCAACTATCAGAGGTAGTTGGCGTCATC
GAGCGCCATCTCGAACCGACGTTGCTGGCCGTACATTTGTACGGCTCCGCAGTGGATGGC
GGCCTGAAGCCACACAGTGATATTGATTTGCTGGTTACGGTGACCGTAAGGCTTGATGAA
ACAACGCGGCGAGCTTTGATCAACGACCTTTTGGAAACTTCGGCTTCCCCTGGAGAGAGC
GAGATTCTCCGCGCTGTAGAAGTCACCATTGTTGTGCACGACGACATCATTCCGTGGCGT
TATCCAGCTAAGCGCGAACTGCAATTTGGAGAATGGCAGCGCAATGACATTCTTGCAGGT
ATCTTCGAGCCAGCCACGATCGACATTGATCTGGCTATCTTGCTGACAAAAGCAAGAGAA
CATAGCGTTGCCTTGGTAGGTCCAGCGGCGGAGGAACTCTTTGATCCGGTTCCTGAACAG
GATCTATTTGAGGCGCTAAATGAAACCTTAACGCTATGGAACTCGCCGCCCGACTGGGCT


# Now, filter reads with kat

In [26]:
threads=16
conda run --name kat /usr/bin/time -v kat filter seq -t ${threads} -o kat.filter.kmer -m 31 \
    --seq input/SRR1952908_1.fastq --seq2 input/SRR1952908_2.fastq \
    staramr-out/aadA1.fasta

Kmer Analysis Toolkit (KAT) V2.4.2

Running KAT in filter sequence mode
-----------------------------------

Input 57904 is a sequence file.  Counting kmers for input 57904 (staramr-out/aadA1.fasta) ... done.  Time taken: 0.1s

Filtering sequences ...
Processed 100000 pairs
Processed 200000 pairs
Processed 300000 pairs
Processed 400000 pairs
Processed 500000 pairs
Processed 600000 pairs
Processed 700000 pairs
Processed 800000 pairs
Processed 900000 pairs
Processed 1000000 pairs
Processed 1100000 pairs
Processed 1200000 pairs
Processed 1300000 pairs
Processed 1400000 pairs
Processed 1500000 pairs
Processed 1600000 pairs
Processed 1700000 pairs
Processed 1800000 pairs
Processed 1900000 pairs
Processed 2000000 pairs
Processed 2100000 pairs
Finished filtering.  Time taken: 107.2s

Found 1224 / 2147046 to keep

KAT filter seq completed.
Total runtime: 107.4s

	Command being timed: "kat filter seq -t 16 -o kat.filter.kmer -m 31 --seq input/SRR1952908_1.fastq --seq2 input/SRR1952908_2.fastq s

In [28]:
ls -lrth kat.filter.kmer*

-rw-r--r-- 1 apetkau grp_apetkau 277K Jan 15 22:44 kat.filter.kmer.in.R2.fastq
-rw-r--r-- 1 apetkau grp_apetkau 278K Jan 15 22:44 kat.filter.kmer.in.R1.fastq


# Assemble extracted reads

In [8]:
conda run --name skesa skesa --help


General options:
  -h [ --help ]                 Produce help message
  -v [ --version ]              Print version
  --cores arg (=0)              Number of cores to use (default all) [integer]
  --memory arg (=32)            Memory available (GB, only for sorted counter) 
                                [integer]
  --hash_count                  Use hash counter [flag]
  --estimated_kmers arg (=100)  Estimated number of unique kmers for bloom 
                                filter (M, only for hash counter) [integer]
  --skip_bloom_filter           Don't do bloom filter; use --estimated_kmers as
                                the hash table size (only for hash counter) 
                                [flag]

Input/output options : at least one input providing reads for assembly must be specified:
  --fasta arg                   Input fasta file(s) (could be used multiple 
                                times for different runs) [string]
  --fastq arg                   Input fastq

In [11]:
conda run --name skesa /usr/bin/time -v skesa --cores ${threads} \
    --fastq kat.filter.kmer.in.R1.fastq,kat.filter.kmer.in.R2.fastq \
    --contigs_out kat.filter.contigs.fasta --vector_percent 1

skesa --cores 16 --fastq kat.filter.kmer.in.R1.fastq,kat.filter.kmer.in.R2.fastq --contigs_out kat.filter.contigs.fasta --vector_percent 1 

Total mates: 2448 Paired reads: 1224
Reads acquired in  0.005521s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)
Adapters clip is disabled

Kmer len: 21
Raw kmers: 193740 Memory needed (GB): 0.00371981 Memory available (GB): 29.9999 1 cycle(s) will be performed
Distinct kmers: 3446
Kmer count in  0.021522s wall, 0.100000s user + 0.030000s system = 0.130000s CPU (604.0%)
Uniq kmers merging in  0.007879s wall, 0.010000s user + 0.010000s system = 0.020000s CPU (253.8%)
Kmers branching in  0.001827s wall, 0.010000s user + 0.000000s system = 0.010000s CPU (547.3%)

Average read length: 99
Genome size estimate: 963

Kmer: 21 Graph size: 2958 Contigs in: 0
Valley: 0

Mark used kmers in  0.000010s wall, 0.000000s user + 0.000000s system = 0.000000s CPU (n/a%)
Kmers in multiple/single contigs: 0 0
Fragments before: 39 2642
Fragments after: 

In [12]:
ls -lh *.fasta

-rw-r--r-- 1 apetkau grp_apetkau 2.7K Jan 15 22:54 kat.filter.contigs.fasta


In [15]:
conda run --name staramr staramr search -o kat-staramr-out kat.filter.contigs.fasta

2020-01-15 22:55:32 INFO: No --pointfinder-organism specified. Will not search the PointFinder databases
2020-01-15 22:55:32 INFO: No --plasmidfinder-database-type specified. Will search the entire PlasmidFinder database
2020-01-15 22:55:32 INFO: --output-dir set. All files will be output to [kat-staramr-out]
2020-01-15 22:55:32 INFO: Will exclude ResFinder/PointFinder genes listed in [/home/CSCScience.ca/apetkau/miniconda3/envs/staramr/lib/python3.7/site-packages/staramr/databases/exclude/data/genes_to_exclude.tsv]. Use --no-exclude-genes to disable
2020-01-15 22:55:32 INFO: Making BLAST databases for input files
2020-01-15 22:55:32 INFO: Scheduling blasts and MLST for kat.filter.contigs.fasta
2020-01-15 22:55:32 INFO: Finished. Took 0.01 minutes.
2020-01-15 22:55:32 INFO: Predicting AMR resistance phenotypes is enabled. The predictions are for microbiological resistance and *not* clinical resistance. These results are continually being improved and we welcome any feedback.
2020-01-15

# Compare results (detected AMR)

In [16]:
cat kat-staramr-out/resfinder.tsv

Isolate ID	Gene	Predicted Phenotype	%Identity	%Overlap	HSP Length/Total Length	Contig	Start	End	Accession	Sequence
kat.filter.contigs	aadA1	streptomycin	100.00	100.00	792/792	Contig_2_98.9478	1219	428	JQ414041	ATGAGGGAAGCGGTGATCGCCGAAGTATCGACTCAACTATCAGAGGTAGTTGGCGTCATCGAGCGCCATCTCGAACCGACGTTGCTGGCCGTACATTTGTACGGCTCCGCAGTGGATGGCGGCCTGAAGCCACACAGTGATATTGATTTGCTGGTTACGGTGACCGTAAGGCTTGATGAAACAACGCGGCGAGCTTTGATCAACGACCTTTTGGAAACTTCGGCTTCCCCTGGAGAGAGCGAGATTCTCCGCGCTGTAGAAGTCACCATTGTTGTGCACGACGACATCATTCCGTGGCGTTATCCAGCTAAGCGCGAACTGCAATTTGGAGAATGGCAGCGCAATGACATTCTTGCAGGTATCTTCGAGCCAGCCACGATCGACATTGATCTGGCTATCTTGCTGACAAAAGCAAGAGAACATAGCGTTGCCTTGGTAGGTCCAGCGGCGGAGGAACTCTTTGATCCGGTTCCTGAACAGGATCTATTTGAGGCGCTAAATGAAACCTTAACGCTATGGAACTCGCCGCCCGACTGGGCTGGCGATGAGCGAAATGTAGTGCTTACGTTGTCCCGCATTTGGTACAGCGCAGTAACCGGCAAAATCGCGCCGAAGGATGTCGCTGCCGACTGGGCAATGGAGCGCCTGCCGGCCCAGTATCAGCCCGTCATACTTGAAGCTAGACAGGCTTATCTTGGACAAGAAGAAGATCGCTTGGCCTCCCGCGCAGATCAGTTGGAAGAATTTGTTCACTACGTGAAAGGCGAGATCACCAAGGTAGTCGGCAAAT

In [17]:
cat staramr-out/resfinder.tsv

Isolate ID	Gene	Predicted Phenotype	%Identity	%Overlap	HSP Length/Total Length	Contig	Start	End	Accession	Sequence
SRR1952908	aadA1	streptomycin	100.00	100.00	792/792	contig00030	5355	4564	JQ414041	ATGAGGGAAGCGGTGATCGCCGAAGTATCGACTCAACTATCAGAGGTAGTTGGCGTCATCGAGCGCCATCTCGAACCGACGTTGCTGGCCGTACATTTGTACGGCTCCGCAGTGGATGGCGGCCTGAAGCCACACAGTGATATTGATTTGCTGGTTACGGTGACCGTAAGGCTTGATGAAACAACGCGGCGAGCTTTGATCAACGACCTTTTGGAAACTTCGGCTTCCCCTGGAGAGAGCGAGATTCTCCGCGCTGTAGAAGTCACCATTGTTGTGCACGACGACATCATTCCGTGGCGTTATCCAGCTAAGCGCGAACTGCAATTTGGAGAATGGCAGCGCAATGACATTCTTGCAGGTATCTTCGAGCCAGCCACGATCGACATTGATCTGGCTATCTTGCTGACAAAAGCAAGAGAACATAGCGTTGCCTTGGTAGGTCCAGCGGCGGAGGAACTCTTTGATCCGGTTCCTGAACAGGATCTATTTGAGGCGCTAAATGAAACCTTAACGCTATGGAACTCGCCGCCCGACTGGGCTGGCGATGAGCGAAATGTAGTGCTTACGTTGTCCCGCATTTGGTACAGCGCAGTAACCGGCAAAATCGCGCCGAAGGATGTCGCTGCCGACTGGGCAATGGAGCGCCTGCCGGCCCAGTATCAGCCCGTCATACTTGAAGCTAGACAGGCTTATCTTGGACAAGAAGAAGATCGCTTGGCCTCCCGCGCAGATCAGTTGGAAGAATTTGTTCACTACGTGAAAGGCGAGATCACCAAGGTAGTCGGCAAATAA
SRR195290